# PRELIMINARY DATA EXPLORATION & CLEANING

In this file I will perform some quick data exploration, basic checks and cleaning after web scraping to ensure that the cars' data has been scraped correctly and is ready for being loaded into the database. I have decided not to make a pipeline, as I am likely not going to download and reprocess the dataset very often. If new car models were to be added to the site, they could have technical specs not prevously included in the table, meaning I would have to go over the whole process manually anyway.

In [ ]:
from database import connect, insert_table
import pandas as pd
from pathlib import Path

# Car technical data

In [714]:
paths = Path("../data/car_technical_data/").glob("*.json")

tech_specs = pd.concat([pd.read_json(p) for p in paths])
tech_specs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13761 entries, 0 to 789
Columns: 158 entries, Model to Przestrzeń wsiadania z tyłu - wysokość
dtypes: float64(96), object(62)
memory usage: 16.7+ MB


In [716]:
display_settings = ['display.max_rows', None, 'display.min_rows', None, 
                       'display.max_columns', None,'display.max_colwidth', None]

with pd.option_context(*display_settings):
    display(tech_specs.head())

,Model,Liczba drzwi,Liczba miejsc,Średnica zawracania,Promień skrętu,Długość,Szerokość z lusterkami bocznymi,Wysokość,Rozstaw osi,Rozstaw kół - przód,Rozstaw kół - tył,Zwis przedni,Zwis tylny,Prześwit,Odległość od siedziska przedniego do dachu,Odległość od siedziska tylnego do dachu,Szerokość nad podłokietnikami z przodu,Szerokość nad podłokietnikami z tyłu,Maksymalna pojemność bagażnika (siedzenia złożone),Minimalna pojemność bagażnika (siedzenia rozłożone),Produkowany,Pojemność skokowa,Typ silnika,Moc silnika,Maksymalny moment obrotowy,Montaż silnika,Doładowanie,Umiejscowienie wałka rozrządu,Liczba cylindrów,Układ cylindrów,Liczba zaworów,Stopień sprężania,Średnica cylindra × skok tłoka,Zapłon,Typ wtrysku,Rodzaj układu kierowniczego,Opony podstawowe,Opony opcjonalne,Rodzaj hamulców (przód),Rodzaj hamulców (tył),Rodzaj zawieszenia (przód),Rodzaj zawieszenia (tył),Amortyzatory,Rodzaj skrzyni,Liczba stopni,Rodzaj napędu,Nazwa skrzyni,Prędkość maksymalna,Przyspieszenie (od 0 do 100km/h),Średnie spalanie (cykl mieszany),Spalanie na trasie (na autostradzie),Spalanie w mieście,Pojemność zbiornika paliwa,Zasięg (cykl mieszany),Zasięg (autostrada),Zasięg (miasto),Emisja CO₂,Norma emisji spalin,Minimalna masa własna pojazdu (bez obciążenia),System start&stop,Średnica tarcz hamulcowych (przód),Średnica tarcz hamulcowych (tył),Liczba biegów,Przyspieszenie (0 - 100 km/h),Spalanie przy 90 km/h,Spalanie przy 140 km/h,Rodzaj sprzęgła,Szerokość,400 metrów ze startu zatrzymanego,1000 metrów ze startu zatrzymanego,Maksymalna masa całkowita pojazdu (w pełni obciążonego),Moc silnika (elektryczny),Grubość tarcz hamulcowych (przód),Grubość tarcz hamulcowych (tył),Pojemność akumulatora netto,Pojemność akumulatora brutto,Moc silnika (spalinowy),Maksymalny moment obrotowy (spalinowy),Liczba silników,Metodologia pomiaru zasięgu,Zużycie energii,Maksymalny zasięg przy oszczędnej jeździe na długiej trasie,Dopuszczalne obciążenie dachu,Maksymalna masa przyczepy z hamulcami,Maksymalna masa przyczepy bez hamulców,Kod silnika,Układ paliwowy,Szerokość ze złożonymi lusterkami bocznymi,Na postoju,Przyspieszanie od 0 do 90 km/h,Przy prędkości 130 km/h,Maksymalny moment obrotowy (elektryczny),Felgi podstawowe,Rozstaw śrub,Pojemność akumulatora,Pojemność akumulatora w wersji z klimatyzacja,Felgi opcjonalne,Maksymalny nacisk na hak,Dodatkowe informacje,Kąt natarcia,Kąt zejścia,Szerokość na wysokości podłokietników z przodu,Szerokość na wysokości podłokietników z tyłu,Szerokość pomiędzy nadkolami,Długość do oparcia tylnej kanapy,Wysokość progu załadowczego,Szerokość bagażnika,Typ układu hamulcowego,Wysokość bagażnika,Długość ze złożoną tylną kanapą,Długość z hakiem holowniczym,Wysokość z relingami dachowymi,Prześwit 4x4,Odległość oparcia przedniego od komory silnika,Długość siedziska przedniego,Odległość pomiędzy siedzeniami przednimi i tylnymi,Długość siedziska tylnego,Typ ładowarki,Chłodzenie akumulatora,Średni maksymalny zasięg,Stacja szybkiego ładowania,Gniazdko 3F/Stacja AC,Gniazdko 1F,Średni minimalny zasięg,Głębokość brodzenia,Wysokość przy otwartej klapie bagażnika,Wysokość siedziska przedniego,Hamowanie (100 do 0km/h) z ABS,Kąt rampowy,Maksymalna moc ładowania DC,Maksymalna ładowność,Wysokość z anteną,Przestrzeń wsiadania z przodu - szerokość,Zakres przesuwania foteli przednich,Odległość oparcia fotela przedniego od kierownicy,Długość kolumny kierownicy,Maksymalna moc ładowania AC,Odległość od podłogi do siedziska tylnego,Całkowita długość wnętrza kabiny,Wysokość oparcia przedniego,Wysokość oparcia tylnego,Drzwi tylne - szerokość,Drzwi tylne - wysokość,Drzwi boczne - szerokość,Drzwi boczne - wysokość,Przedział ładunkowy - długość,Przedział ładunkowy - szerokość,Przedział ładunkowy - wysokość,Kąt przechyłu bocznego,Całkowita wysokość wnętrza kabiny,Całkowita szerokość wnętrza kabiny,Szerokość przy otwartych drzwiach z przodu,Możliwość podjazdu,Wysokość przy otwartej pokrywie silnika,Przestrzeń wsiadania z przodu - wysokość,Szerokość przy otwartych drzwiach z

That is a lot of data in a large variety of forms. A considerable amout of cleaning will be needed.

In [717]:
print('Columns with less than 1000 non-null values: ', (tech_specs.notnull().sum() < 1000).sum())

Columns with less than 1000 non-null values:  70


Lots of missing values, let's dive deeper into these columns and decide which ones might still have value for us

##### Filtering the data

In [718]:
with pd.option_context(*display_settings):
    not_null_counts = tech_specs.notnull().sum()
    display(not_null_counts[not_null_counts < 1000])

Przyspieszenie (0 - 100 km/h)                                  135
Spalanie przy 90 km/h                                           93
Spalanie przy 140 km/h                                          85
400 metrów ze startu zatrzymanego                              222
1000 metrów ze startu zatrzymanego                             656
Moc silnika (elektryczny)                                      117
Grubość tarcz hamulcowych (przód)                              571
Grubość tarcz hamulcowych (tył)                                509
Pojemność akumulatora netto                                     86
Pojemność akumulatora brutto                                   326
Moc silnika (spalinowy)                                        125
Maksymalny moment obrotowy (spalinowy)                         119
Liczba silników                                                137
Metodologia pomiaru zasięgu                                    195
Zużycie energii                                               

Columns I have decided to keep in the dataset and reasons why:


| Column name                                                   | Reason                                                                                     |
|---------------------------------------------------------------|--------------------------------------------------------------------------------------------|
| Przyspieszenie (0 - 100 km/h)                                 | it's a very important metric, no idea why it was skipped so often                          |
| Szerokość ze złożonymi lusterkami bocznymi                    | gives more insight into needed parking space size                                          |
| Pojemność akumulatora                                         | could be useful someday                                                                    |
| Felgi opcjonalne                                              | useful if somebody wants to check new wheel/tire fitment options                           |
| Długość z hakiem holowniczym                                  | few cars have tow hooks as a factory option, nulls were expected                           |
| Wysokość z relingami dachowymi                                | same as above but with roof rack mounts, nulls were expected                               |
| Prześwit 4x4                                                  | same as above, key metric for off-roaders                                                  |
| Głębokość brodzenia                                           | same as above, useful for off-roaders                                                      |
| Wysokość przy otwartej klapie bagażnika                       | same as above, useful for garage size                                                      |
| Kąt rampowy                                                   | same as above, useful for off-roaders                                                      |
| Maksymalna ładowność                                          | same as above, key metric for utility vehicles                                             |
| Odległość oparcia fotela przedniego od kierownicy             | could be useful for disabled people                                                        |
| Przedział ładunkowy - długość                                 | key metric for utility vehicles                                                            |
| Przedział ładunkowy - szerokość                               | same as above                                                                              |
| Przedział ładunkowy - wysokość                                | same as above                                                                              |


In [719]:
high_null_cols_keep = ['Przyspieszenie (0 - 100 km/h)', 
                        'Szerokość ze złożonymi lusterkami bocznymi',
                        'Pojemność akumulatora',
                        'Felgi opcjonalne',
                        'Długość z hakiem holowniczym',
                        'Wysokość z relingami dachowymi',
                        'Prześwit 4x4',
                        'Głębokość brodzenia',
                        'Wysokość przy otwartej klapie bagażnika',
                        'Kąt rampowy',
                        'Maksymalna ładowność',
                        'Odległość oparcia fotela przedniego od kierownicy',
                        'Przedział ładunkowy - długość',
                        'Przedział ładunkowy - szerokość',
                        'Przedział ładunkowy - wysokość']

In [720]:
columns_keep = not_null_counts[not_null_counts >= 1000].index.tolist() + high_null_cols_keep

tech_specs_useful = tech_specs[columns_keep]
len(tech_specs_useful.columns)

103

##### Exploring unique values

In [721]:
with pd.option_context(*display_settings):
    display(tech_specs_useful.nunique().sort_values())

Amortyzatory                                                   3
Rodzaj napędu                                                  3
Zapłon                                                         3
Rodzaj skrzyni                                                 4
Rodzaj sprzęgła                                                4
Liczba drzwi                                                   5
Układ cylindrów                                                5
Typ silnika                                                    5
Rodzaj hamulców (przód)                                        5
Rodzaj hamulców (tył)                                          6
Umiejscowienie wałka rozrządu                                  6
Montaż silnika                                                 6
Doładowanie                                                    7
Liczba biegów                                                  8
Liczba miejsc                                                  8
Liczba cylindrów         

In [722]:
tech_specs_useful['Opony podstawowe'].unique()

array(['225/40 R19; 225/45 R18', '225/50 R17', '245/35 R19', '205/60 R16',
       nan, '155/60 R15', '215/55 R18', '225/40 R20', '235/60 R18',
       '225/65 R17; 235/55 R19', '225/65 R17', '235/50 R18', '185/65 R15',
       '205/55 R16', '205/60 R15', '205/55 R15', '195/65 R15',
       '215/55 R16', '205/50 R16', '165/70 R13', '185/60 R14',
       '175/70 R13', '225/45 R17', '185/70 R13', '195/60 R15',
       '185/70 R14', '195/55 R16', '215/45 R17', '165/65 R14',
       '185/60 R15', '225/55 R18', '225/55 R17', '225/60 R17',
       '225/60 R16', '255/55 R17', '215/55 R17', '225/55 R16',
       '235/55 R17', '255/45 R18', '235/55 R18', '255/45 R19',
       '235/60 R17', '235/60 R16', '245/45 R18', '255/55 R19',
       '215/65 R17', '215/65 R16', '245/30 R21', '235/65 R17',
       '255/60 R18', '255/55 R18', '265/55 R19', '285/45 R21',
       '235/40 R18', '235/35 R19', '205/50 R17', '245/40 R18',
       '245/45 R17', '285/45 R18', '255/50 R18', '235/65 R16',
       '245/45 R19', '245/

We have lots of categorical values, so I won't do One-Hot encoding yet, maybe in the future if I decide to train some model on this data (most probably clustering). One thing I will do now is create an unique key for each model and create two more tables - wheel size and tire size. It would be nice to know which cars can interchange tires.

##### Creating the primary key

In [723]:
for model in tech_specs_useful['Model'].iloc[[0,1000,2000,3000,4000,5000]]:
    print(model)

alfa-romeo giulia ii sedan silnik-benzynowy-2.0-tbi-280km-2016-2019
bmw seria-5 e39 touring silnik-benzynowy-523-i-170km-1997-2004
ds 3 hatchback-facelifting-2014-citroen silnik-diesla-1.6-hdi-92km-2014-2015
ford tourneo-connect ii grand silnik-diesla-1.6-duratorq-tdci-115km-2013-2018
hyundai coupe iii silnik-benzynowy-2.0-i-16v-138km-2002-2003
mazda mpv i silnik-benzynowy-3.0-i-v6-148km-1990-1999


In [724]:
name_lenghts = set()

for model in tech_specs_useful['Model']:
    name_lenghts.add(len(model.split(' ')))

print(name_lenghts)

{4, 5}


Looks like some cars have body type included in their model name, while others don't

In [ ]:
def create_model_name_columns(name):
    name_parts = name.split(' ')
    if len(name_parts) == 4:
        name_parts.insert(3, None)
    return pd.Series(name_parts)

In [726]:
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

id_cols = ['Marka', 'Model', 'Generacja', 'Nadwozie', 'Silnik'] # These will become the primary key in the database
tech_specs_useful[id_cols] = tech_specs_useful['Model'].apply(create_model_name_columns)

other_cols = [col for col in tech_specs_useful.columns if col not in id_cols]
new_col_order = id_cols + other_cols
tech_specs_useful = tech_specs_useful[new_col_order]

In [727]:
tech_specs_useful.columns[:10]

Index(['Marka', 'Model', 'Generacja', 'Nadwozie', 'Silnik', 'Liczba drzwi',
       'Liczba miejsc', 'Średnica zawracania', 'Promień skrętu', 'Długość'],
      dtype='object')

In [728]:
print(tech_specs_useful.columns.tolist())  # Print first 10 columns to check

['Marka', 'Model', 'Generacja', 'Nadwozie', 'Silnik', 'Liczba drzwi', 'Liczba miejsc', 'Średnica zawracania', 'Promień skrętu', 'Długość', 'Szerokość z lusterkami bocznymi', 'Wysokość', 'Rozstaw osi', 'Rozstaw kół - przód', 'Rozstaw kół - tył', 'Zwis przedni', 'Zwis tylny', 'Prześwit', 'Odległość od siedziska przedniego do dachu', 'Odległość od siedziska tylnego do dachu', 'Szerokość nad podłokietnikami z przodu', 'Szerokość nad podłokietnikami z tyłu', 'Maksymalna pojemność bagażnika (siedzenia złożone)', 'Minimalna pojemność bagażnika (siedzenia rozłożone)', 'Produkowany', 'Pojemność skokowa', 'Typ silnika', 'Moc silnika', 'Maksymalny moment obrotowy', 'Montaż silnika', 'Doładowanie', 'Umiejscowienie wałka rozrządu', 'Liczba cylindrów', 'Układ cylindrów', 'Liczba zaworów', 'Stopień sprężania', 'Średnica cylindra × skok tłoka', 'Zapłon', 'Typ wtrysku', 'Rodzaj układu kierowniczego', 'Opony podstawowe', 'Opony opcjonalne', 'Rodzaj hamulców (przód)', 'Rodzaj hamulców (tył)', 'Rodzaj zaw

##### Renaming columns for better clarity

In [729]:
new_col_names =['brand',
            'model',
            'generation',
            'body_type',
            'engine',
            'number_of_doors',
            'number_of_seats',
            'turning_diameter',
            'turning_radius',
            'length',
            'width_with_mirrors',
            'height',
            'wheelbase',
            'front_track_width',
            'rear_track_width',
            'front_overhang',
            'rear_overhang',
            'ground_clearance',
            'front_seat_to_roof_distance',
            'rear_seat_to_roof_distance',
            'front_width_above_armrests',
            'rear_width_above_armrests',
            'max_trunk_capacity_folded',
            'min_trunk_capacity_unfolded',
            'production_years',
            'engine_displacement',
            'engine_type',
            'engine_power',
            'max_torque',
            'engine_mounting',
            'turbocharging',
            'camshaft_location',
            'number_of_cylinders',
            'cylinder_layout',
            'number_of_valves',
            'compression_ratio',
            'bore_x_stroke',
            'ignition_type',
            'fuel_injection_type',
            'steering_system_type',
            'standard_tires',
            'optional_tires',
            'front_brake_type',
            'rear_brake_type',
            'front_suspension_type',
            'rear_suspension_type',
            'shock_absorbers',
            'transmission_type',
            'number_of_gears',
            'drive_type',
            'gearbox_name',
            'top_speed',
            'acceleration_0_100',
            'avg_fuel_consumption_combined',
            'fuel_consumption_highway',
            'fuel_consumption_city',
            'fuel_tank_capacity',
            'range_combined',
            'range_highway',
            'range_city',
            'co2_emissions',
            'emission_standard',
            'curb_weight_min',
            'start_stop_system',
            'front_brake_disc_diameter',
            'rear_brake_disc_diameter',
            'gear_count',
            'clutch_type',
            'width',
            'gross_vehicle_weight',
            'roof_load_limit',
            'max_trailer_weight_braked',
            'max_trailer_weight_unbraked',
            'engine_code',
            'standard_rims',
            'bolt_pattern',
            'max_towbar_load',
            'approach_angle',
            'departure_angle',
            'front_armrest_width',
            'rear_armrest_width',
            'width_between_wheel_arches',
            'cargo_length_to_rear_seat',
            'loading_sill_height',
            'trunk_width',
            'trunk_height',
            'cargo_length_with_rear_seats_folded',
            'distance_front_seat_to_engine_bay',
            'front_seat_cushion_length',
            'distance_between_front_and_rear_seats',
            'rear_seat_cushion_length',
            'total_cabin_length',
            'acceleration_0_100_alt',
            'width_with_folded_mirrors',
            'battery_capacity',
            'optional_rims',
            'length_with_towbar',
            'height_with_roof_rails',
            'ground_clearance_4x4',
            'wading_depth',
            'height_with_tailgate_open',
            'breakover_angle',
            'max_payload',
            'front_seatback_to_steering_wheel_distance',
            'cargo_bay_length',
            'cargo_bay_width',
            'cargo_bay_height']


In [730]:
tech_specs_useful = tech_specs_useful.set_axis(new_col_names, axis=1)

##### Creating the wheel and tire tables

In [731]:
with pd.option_context(*display_settings):
    display(tech_specs_useful.head())

,brand,model,generation,body_type,engine,number_of_doors,number_of_seats,turning_diameter,turning_radius,length,width_with_mirrors,height,wheelbase,front_track_width,rear_track_width,front_overhang,rear_overhang,ground_clearance,front_seat_to_roof_distance,rear_seat_to_roof_distance,front_width_above_armrests,rear_width_above_armrests,max_trunk_capacity_folded,min_trunk_capacity_unfolded,production_years,engine_displacement,engine_type,engine_power,max_torque,engine_mounting,turbocharging,camshaft_location,number_of_cylinders,cylinder_layout,number_of_valves,compression_ratio,bore_x_stroke,ignition_type,fuel_injection_type,steering_system_type,standard_tires,optional_tires,front_brake_type,rear_brake_type,front_suspension_type,rear_suspension_type,shock_absorbers,transmission_type,number_of_gears,drive_type,gearbox_name,top_speed,acceleration_0_100,avg_fuel_consumption_combined,fuel_consumption_highway,fuel_consumption_city,fuel_tank_capacity,range_combined,range_highway,range_city,co2_emissions,emission_standard,curb_weight_min,start_stop_system,front_brake_disc_diameter,rear_brake_disc_diameter,gear_count,clutch_type,width,gross_vehicle_weight,roof_load_limit,max_trailer_weight_braked,max_trailer_weight_unbraked,engine_code,standard_rims,bolt_pattern,max_towbar_load,approach_angle,departure_angle,front_armrest_width,rear_armrest_width,width_between_wheel_arches,cargo_length_to_rear_seat,loading_sill_height,trunk_width,trunk_height,cargo_length_with_rear_seats_folded,distance_front_seat_to_engine_bay,front_seat_cushion_length,distance_between_front_and_rear_seats,rear_seat_cushion_length,total_cabin_length,acceleration_0_100_alt,width_with_folded_mirrors,battery_capacity,optional_rims,length_with_towbar,height_with_roof_rails,ground_clearance_4x4,wading_depth,height_with_tailgate_open,breakover_angle,max_payload,front_seatback_to_steering_wheel_distance,cargo_bay_length,cargo_bay_width,cargo_bay_height
0,alfa-romeo,giulia,ii,sedan,silnik-benzynowy-2.0-tbi-280km-2016-2019,4.0,5.0,10.8 m,5.4 m,4637.0,2021.0,1425.0,2819.0,1559.0,1604.0,795.0,1028.0,152.0,955.0,955.0,1425.0,1362.0,1500.0,480.0,od 2016 do 2019 roku,1995.0,benzynowy,280KM(206kW) przy 5250 obr/min,400Nmprzy 2250 obr/min,"z przodu, podłużnie",turbosprężarkowe,DOHC,4.0,rzędowy,16.0,10.0 : 1,84 × 90,elektroniczny,bezpośredni,przekładnia zębatkowa ze wspomaganiem elektrycznym,225/40 R19; 225/45 R18,225/35 R19,tarczowe wentylowane,tarczowe wentylowane,system czworoboku wysokiego,system o architekturze Multilink,gazowe,automatyczna,8.0,4x4,AT8,240.0,5.0,7.0,6.0,9.0,58.0,829.0,967.0,644.0,175 g/km,Euro 6,1530.0,dostępny,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,alfa-romeo,giulia,ii,sedan,silnik-benzynowy-2.0-turbo-200km-2015-2019,4.0,5.0,10.8 m,5.4 m,4637.0,2021.0,1425.0,2819.0,1559.0,1604.0,795.0,1028.0,152.0,955.0,955.0,1425.0,1362.0,1500.0,480.0,od 2015 do 2019 roku,1995.0,benzynowy,200KM(147kW) przy 5000 obr/min,330Nmprzy 1750 obr/min,"z przodu, podłużnie",turbodoładowanie o zmiennej geometrii (Variable Geometry Turbo),DOHC,4.0,rzędowy,16.0,10.0 : 1,84 × 90,elektroniczny,bezpośredni,przekładnia zębatkowa ze wspomaganiem elektrycznym,225/50 R17,225/40 R19; 225/45 R18,tarczowe wentylowane,tarczowe wentylowane,wielowahaczowe double wishbone,system o architekturze Multilink,gazowe,automatyczna,8.0,na tylną oś,AT8,235.0,6.0,6.0,5.0,8.0,58.0,967.0,1160.0,725.0,152 g/km,Euro 6,1429.0,dostępny,305.0,292.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,alfa-romeo,giulia,ii,sedan,silnik-benzynowy-2.9-v6-510km-2015-2019,4.0,5.0,10.8 m,5.4 m,4637.0,2021.0,1425.0,2819.0,1559.0,1604.0,795.0,1028.0,152.0,955.0,955.0,1425.0,1362.0,1500.0,480.0,od 2015 do 2019 roku,2891.0,benzynowy,510KM(375kW) przy 6500 obr/min,600Nmprzy 2500-5500 obr/min

In [732]:
import re

In [733]:
tire_specs = tech_specs_useful[['brand', 'model', 'generation', 'body_type', 'engine', 'standard_tires', 'optional_tires']]
tire_specs.head()

,brand,model,generation,body_type,engine,standard_tires,optional_tires
0,alfa-romeo,giulia,ii,sedan,silnik-benzynowy-2.0-tbi-280km-2016-2019,225/40 R19; 225/45 R18,225/35 R19
1,alfa-romeo,giulia,ii,sedan,silnik-benzynowy-2.0-turbo-200km-2015-2019,225/50 R17,225/40 R19; 225/45 R18
2,alfa-romeo,giulia,ii,sedan,silnik-benzynowy-2.9-v6-510km-2015-2019,245/35 R19,225/45 R19; 245/40 R19
3,alfa-romeo,giulia,ii,sedan,silnik-diesla-22-jtdm-2-160km-2018-2019,205/60 R16,225/40 R19; 225/45 R18; 225/50 R17
4,alfa-romeo,giulia,ii,sedan,silnik-diesla-22-jtdm-2-190km-2018-2019,205/60 R16,225/40 R19; 225/45 R18; 225/50 R17


In [734]:
def missing_val_fixer(std, opt):
    # If it's NaN or None treat as empty list
    if not isinstance(std, list):
        std = [] if pd.isna(std) else [std]
    if not isinstance(opt, list):
        opt = [] if pd.isna(opt) else [opt]
    return std + opt

In [735]:
tire_specs['standard_tires'] = tire_specs['standard_tires'].str.split(r';\s*')
tire_specs['optional_tires'] = tire_specs['optional_tires'].str.replace('R', 'RO').str.split(r';\s*')

tire_specs['tire'] = tire_specs.apply(lambda row: missing_val_fixer(row['standard_tires'], row['optional_tires']), axis=1)
tire_specs = tire_specs.explode('tire')

tire_pattern = re.compile(r'(\d{3})/(\d{2})\s*R(O\d{2}|\d{2})')
tire_specs[['tire_width', 'side_profile', 'wheel_size']] = tire_specs['tire'].str.extract(tire_pattern)

tire_specs['is_standard'] = tire_specs['wheel_size'].apply(lambda x: True if str(x)[0] != 'O' else False)
tire_specs['wheel_size'] = tire_specs['wheel_size'].str.replace('O', '')

tire_specs.drop(columns=['standard_tires', 'optional_tires', 'tire'], inplace=True)
tire_specs.head()

,brand,model,generation,body_type,engine,tire_width,side_profile,wheel_size,is_standard
0,alfa-romeo,giulia,ii,sedan,silnik-benzynowy-2.0-tbi-280km-2016-2019,225,40,19,True
0,alfa-romeo,giulia,ii,sedan,silnik-benzynowy-2.0-tbi-280km-2016-2019,225,45,18,True
0,alfa-romeo,giulia,ii,sedan,silnik-benzynowy-2.0-tbi-280km-2016-2019,225,35,19,False
1,alfa-romeo,giulia,ii,sedan,silnik-benzynowy-2.0-turbo-200km-2015-2019,225,50,17,True
1,alfa-romeo,giulia,ii,sedan,silnik-benzynowy-2.0-turbo-200km-2015-2019,225,40,19,False


In [736]:
wheel_specs = tech_specs_useful[['brand', 'model', 'generation', 'body_type', 'engine', 'standard_rims', 'optional_rims', 'bolt_pattern']]
wheel_specs.iloc[1285:1287]

,brand,model,generation,body_type,engine,standard_rims,optional_rims,bolt_pattern
1285,bmw,seria-6,g32,gran-tourismo,silnik-diesla-620d-190km-2018-2020,"7,50Jx17","10,00Jx20; 8,00Jx18; 8,50Jx19; 9,50Jx19",NaN
1286,bmw,seria-6,g32,gran-tourismo,silnik-diesla-630d-265km-2017-2020,7.5J X 17,NaN,NaN


In [737]:
wheel_specs['standard_rims'] = wheel_specs['standard_rims'].str.replace(' ', '').str.replace('J','').str.replace(',','.').str.split(r';\s*')
wheel_specs['optional_rims'] = wheel_specs['optional_rims'].str.replace('J', 'O').str.replace(',','.').str.split(r';\s*')

wheel_specs['rim'] = wheel_specs.apply(lambda row: missing_val_fixer(row['standard_rims'], row['optional_rims']), axis=1)
wheel_specs = wheel_specs.explode('rim')

rim_pattern = re.compile(r'(\d{1,2}\.\d{1,2}O?)[x|X](\d+)')
wheel_specs[['rim_width', 'rim_size']] = wheel_specs['rim'].str.extract(rim_pattern)

wheel_specs['is_standard'] = wheel_specs['rim_width'].apply(lambda x: True if str(x)[-1] != 'O' else False)
wheel_specs['rim_width'] = wheel_specs['rim_width'].str.replace('O', '')

wheel_specs.drop(columns=['standard_rims', 'optional_rims', 'rim'], inplace=True)

wheel_specs.iloc[1384:1390]

,brand,model,generation,body_type,engine,bolt_pattern,rim_width,rim_size,is_standard
1285,bmw,seria-6,g32,gran-tourismo,silnik-diesla-620d-190km-2018-2020,NaN,10.00,20,False
1285,bmw,seria-6,g32,gran-tourismo,silnik-diesla-620d-190km-2018-2020,NaN,8.00,18,False
1285,bmw,seria-6,g32,gran-tourismo,silnik-diesla-620d-190km-2018-2020,NaN,8.50,19,False
1285,bmw,seria-6,g32,gran-tourismo,silnik-diesla-620d-190km-2018-2020,NaN,9.50,19,False
1286,bmw,seria-6,g32,gran-tourismo,silnik-diesla-630d-265km-2017-2020,NaN,7.5,17,True
1287,bmw,seria-6,g32,gran-tourismo,silnik-diesla-640d-320km-2017-2020,NaN,7.50,17,True


In [738]:
tech_specs_useful.drop(columns=['standard_tires', 'optional_tires', 'standard_rims', 'optional_rims'], inplace=True)

##### Grinding the gears

While looking around the tech_specs_useful table i noticed something ambiguous: there are 2 columns that determine how many gears the car has and they are not always equal. By browsing the website I figured that it was the scrapers' fault, as there is usually a choice of available gearboxes for every engine model. I have deemed this issue not important enough to redo the whole web-scraping process, as it took more than 12 hours. I will use the 'gear_count' column as a gear count for manual gearboxes and 'number_of_gears' as gear count for automatic gearboxes, renaming them accordingly. 'gearbox_name' column will remain unchanged (but is not to be trusted in 100%), in place of the 'transmission_type' column there will be 2 columns showing the availability of manual and automatic transmissions.

In [739]:
tech_specs_useful.rename(columns={'gear_count': 'manual_gear_count',
                                 'number_of_gears': 'automatic_gear_count'}, inplace=True)

In [740]:
def gearbox_type_checker(gb_type, gb_numbers, manual):
    if manual:
        if gb_type == 'manualna':
            return True
    else:
        if gb_type is not None and gb_type != 'manualna':
            return True
    if not pd.isna(gb_numbers):
        return True
    return False

In [741]:
tech_specs_useful['manual_gearbox_available'] = tech_specs_useful.apply(lambda row: gearbox_type_checker(row['transmission_type'], row['manual_gear_count'], True), axis=1)
tech_specs_useful['automatic_gearbox_available'] = tech_specs_useful.apply(lambda row: gearbox_type_checker(row['transmission_type'], row['automatic_gear_count'], False), axis=1)
tech_specs_useful.drop('transmission_type', axis=1, inplace=True)

with pd.option_context(*display_settings):
        display(tech_specs_useful[['manual_gear_count',
                                   'automatic_gear_count',
                                   'manual_gearbox_available',
                                   'automatic_gearbox_available']].iloc[38:42])

,manual_gear_count,automatic_gear_count,manual_gearbox_available,automatic_gearbox_available
38,NaN,7.0,False,True
39,5.0,NaN,True,False
40,5.0,NaN,True,False
41,5.0,NaN,True,True


##### Splitting production years

In [742]:
tech_specs_useful['production_years'].notna().sum() # no missing data here

np.int64(13761)

In [743]:
years_start_pattern = re.compile(r'(?:od|w)\s(\d{4})')
years_end_pattern = re.compile(r'(?:do|w)\s(\d{4})') # end_year is NaN if production is ongoing

tech_specs_useful['production_start_year'] = tech_specs_useful['production_years'].str.extract(years_start_pattern)
tech_specs_useful['production_end_year'] = tech_specs_useful['production_years'].str.extract(years_end_pattern)

In [744]:
with pd.option_context(*display_settings):
    display(tech_specs_useful[['production_years', 'production_start_year', 'production_end_year']].iloc[12:18])

tech_specs_useful.drop('production_years', axis=1, inplace=True)

,production_years,production_start_year,production_end_year
12,od 1976 do 1978 roku,1976,1978
13,od 2024 roku,2024,NaN
14,od 2024 roku,2024,NaN
15,od 2024 roku,2024,NaN
16,od 2024 roku,2024,NaN
17,w 2023 roku,2023,2023


##### Extracting engine power figures

In [745]:
tech_specs_useful[['engine_power', 'max_torque']].head()

,engine_power,max_torque
0,280KM(206kW) przy 5250 obr/min,400Nmprzy 2250 obr/min
1,200KM(147kW) przy 5000 obr/min,330Nmprzy 1750 obr/min
2,510KM(375kW) przy 6500 obr/min,600Nmprzy 2500-5500 obr/min
3,160KM(118kW) przy 3250 obr/min,400Nmprzy 1500 obr/min
4,190KM(140kW) przy 3500 obr/min,400Nmprzy 1500 obr/min


In [746]:
tech_specs_useful[['engine_power', 'max_torque']].notnull().sum() # some missing data for 'max_torque' column

engine_power    13761
max_torque      11411
dtype: int64

In [747]:
power_pattern = re.compile(r'(\d+)KM.*?(\d+)\s*obr')
torque_pattern = re.compile(r'(\d+)\s*Nm|M')

tech_specs_useful[['max_hp', 'max_hp_rpm']] = tech_specs_useful['engine_power'].str.extract(power_pattern)
tech_specs_useful['max_torque'] = tech_specs_useful['max_torque'].str.extract(torque_pattern)

tech_specs_useful.drop('engine_power', axis=1, inplace=True)
display(tech_specs_useful[['max_hp', 'max_hp_rpm', 'max_torque']].head())

,max_hp,max_hp_rpm,max_torque
0,280,5250,400
1,200,5000,330
2,510,6500,600
3,160,3250,400
4,190,3500,400


##### Start-Stop system cleanup

In [748]:
tech_specs_useful['start_stop_system'].unique()

array(['dostępny', nan, 'Start&Stop', 'S&S', 'Start-Stop', 'Stop & Start',
       'Start & Stop', 'ASS', 'Auto-Start-Stop', 'BlueDrive',
       'KiaECOdynamics', 'Start/ Stop', 'Start / Stop', 'i-stop',
       'i-STOP', 'i-ELOOP', 'ECO start-stop', 'ECO Start-Stop',
       'BlueEFFICIENCY', 'Start-stop', 'ECO Start-stop', 'Eco start-stop',
       'AS&G', 'Auto Stop & Go', 'Start/Stop', 'STOP & START',
       'auto start/stop', 'Auto Start Stop', 'Auto Start-Stop',
       'stop/start', 'Start Stop', 'Start/Stop Ecomotive', 'Stop & Go',
       'Stop&Go'], dtype=object)

Too many start-stop system names that don't contribute any meaningful information to the table. A simple boolean will be better.

In [749]:
tech_specs_useful['start_stop_system'][4:9]

4    dostępny
5         NaN
6         NaN
7    dostępny
8         NaN
Name: start_stop_system, dtype: object

In [750]:
tech_specs_useful['start_stop_system'] = tech_specs_useful['start_stop_system'].apply(lambda x: True if pd.notna(x) else False)

tech_specs_useful['start_stop_system'][4:9]

4     True
5    False
6    False
7     True
8    False
Name: start_stop_system, dtype: bool

##### Moving units to column names

In [751]:
tech_specs_useful[['turning_diameter', 'turning_radius', 'co2_emissions', 'battery_capacity',
                  'approach_angle', 'departure_angle', 'breakover_angle']].iloc[1818:1820]

,turning_diameter,turning_radius,co2_emissions,battery_capacity,approach_angle,departure_angle,breakover_angle
1818,10.1 m,5.0 m,143 g/km,NaN,30.0 °,34.0 °,21.0 °
1819,10.1 m,5.0 m,145 g/km,60 Ah,30.0 °,34.0 °,21.0 °


In [752]:
tech_specs_useful.rename(columns={'turning_diameter': 'turning_diameter_m',
                                  'turning_radius': 'turning_radius_m',
                                  'co2_emissions': 'co2_emissions_g_per_km',
                                  'approach_angle': 'approach_angle_deg',
                                  'departure_angle': 'departure_angle_deg',
                                  'breakover_angle': 'breakover_angle_deg',
                                  'compression_ratio': 'compression_ratio_to_1',
                                  'battery_capacity': 'battery_capacity_ah'}, inplace=True)

cols = [
    'turning_diameter_m', 'turning_radius_m', 'co2_emissions_g_per_km',
    'battery_capacity_ah', 'approach_angle_deg', 'departure_angle_deg', 
    'breakover_angle_deg', 'compression_ratio_to_1', 
]

for col in cols:
    tech_specs_useful[col] = tech_specs_useful[col].str.split('\xa0').str[0]
tech_specs_useful['compression_ratio_to_1'] = tech_specs_useful['compression_ratio_to_1'].str.split(' ').str[0]

tech_specs_useful.drop('acceleration_0_100_alt', axis=1, inplace=True) # not needed

tech_specs_useful[cols].iloc[1818:1822]

,turning_diameter_m,turning_radius_m,co2_emissions_g_per_km,battery_capacity_ah,approach_angle_deg,departure_angle_deg,breakover_angle_deg,compression_ratio_to_1
1818,10.1,5.0,143,NaN,30.0,34.0,21.0,NaN
1819,10.1,5.0,145,60,30.0,34.0,21.0,NaN
1820,10.1,5.0,173,NaN,30.0,34.0,21.0,NaN
1821,10.1,5.0,158,NaN,30.0,34.0,21.0,NaN


##### Bore & stroke split

Piston bore and stroke should be split into separate columns. Singular values are errors on the site's side, they will be deleted.

In [753]:
tech_specs_useful['bore_x_stroke'].head()

0      84 × 90
1      84 × 90
2    86.5 × 82
3         15.5
4          NaN
Name: bore_x_stroke, dtype: object

In [754]:
piston_match = re.compile(r'(\d*\.?\d*)\s*[x×X✕]\s*(\d*\.?\d*)')

tech_specs_useful[['piston_bore_mm', 'piston_stroke_mm']] = tech_specs_useful['bore_x_stroke'].str.extract(piston_match)
tech_specs_useful.drop('bore_x_stroke', axis=1, inplace=True)

tech_specs_useful[['piston_bore_mm', 'piston_stroke_mm']].head()

,piston_bore_mm,piston_stroke_mm
0,84,90
1,84,90
2,86.5,82
3,NaN,NaN
4,NaN,NaN


##### Setting data types

In [755]:
with pd.option_context(*display_settings):
    display(tech_specs_useful.dtypes)

brand                                         object
model                                         object
generation                                    object
body_type                                     object
engine                                        object
number_of_doors                              float64
number_of_seats                              float64
turning_diameter_m                            object
turning_radius_m                              object
length                                       float64
width_with_mirrors                           float64
height                                       float64
wheelbase                                    float64
front_track_width                            float64
rear_track_width                             float64
front_overhang                               float64
rear_overhang                                float64
ground_clearance                             float64
front_seat_to_roof_distance                  f

Some data types need to be changed.

In [ ]:
to_int_cols = ['number_of_doors', 'number_of_seats', 'manual_gear_count', 'automatic_gear_count',
               'number_of_cylinders', 'number_of_valves', 'production_start_year', 'production_end_year']
to_float_cols = ['turning_diameter_m', 'turning_radius_m', 'compression_ratio_to_1', 'co2_emissions_g_per_km',
               'approach_angle_deg', 'departure_angle_deg', 'breakover_angle_deg', 'battery_capacity_ah',
               'max_hp', 'max_hp_rpm', 'max_torque', 'piston_bore_mm', 'piston_stroke_mm']

for col in to_int_cols:
    tech_specs_useful[col] = pd.to_numeric(tech_specs_useful[col], errors='coerce').astype('Int64')

for col in to_float_cols:
    tech_specs_useful[col] = pd.to_numeric(tech_specs_useful[col], errors='coerce').astype('float64')

with pd.option_context(*display_settings):
    display(tech_specs_useful.dtypes)

brand                                         object
model                                         object
generation                                    object
body_type                                     object
engine                                        object
number_of_doors                                Int64
number_of_seats                                Int64
turning_diameter_m                           float64
turning_radius_m                             float64
length                                       float64
width_with_mirrors                           float64
height                                       float64
wheelbase                                    float64
front_track_width                            float64
rear_track_width                             float64
front_overhang                               float64
rear_overhang                                float64
ground_clearance                             float64
front_seat_to_roof_distance                  f

tire_specs table

In [759]:
tire_specs.dtypes

brand           object
model           object
generation      object
body_type       object
engine          object
tire_width      object
side_profile    object
wheel_size      object
is_standard       bool
dtype: object

In [766]:
for col in ['tire_width', 'side_profile', 'wheel_size']:
    tire_specs[col] = pd.to_numeric(tire_specs[col], errors='coerce')

tire_specs.dtypes

brand            object
model            object
generation       object
body_type        object
engine           object
tire_width      float64
side_profile    float64
wheel_size      float64
is_standard        bool
dtype: object

wheel_specs table

In [764]:
wheel_specs.dtypes

brand           object
model           object
generation      object
body_type       object
engine          object
bolt_pattern    object
rim_width       object
rim_size        object
is_standard       bool
dtype: object

In [767]:
for col in ['rim_width', 'rim_size']:
    wheel_specs[col] = pd.to_numeric(wheel_specs[col], errors='coerce')

wheel_specs.dtypes

brand            object
model            object
generation       object
body_type        object
engine           object
bolt_pattern     object
rim_width       float64
rim_size        float64
is_standard        bool
dtype: object

#### With this the tech_specs_useful, tire_specs and wheel_specs tables are clean and ready for loading nto the database

# Car ratings data

In [792]:
columns = ['model', 'new_price', 'avg_rating', 'buy_again', 'overall_rating', 'engine', 'gearbox', 'drivetrain', 'body', 'visibility',
           'ergonomy', 'climate_control', 'sound_insulation', 'interior_space', 'maintenance_costs', 'quality_price_ratio',
           'reliability_small_repairs', 'reliability_major_repairs']

car_ratings = pd.read_csv('../data/car_ratings.txt', delimiter=';', names=columns)
car_ratings.head()

,model,new_price,avg_rating,buy_again,overall_rating,engine,gearbox,drivetrain,body,visibility,ergonomy,climate_control,sound_insulation,interior_space,maintenance_costs,quality_price_ratio,reliability_small_repairs,reliability_major_repairs
0,alfa-romeo giulia ii,232400.0,4.56,0.98,4.77,4.87,4.83,4.94,4.77,4.02,4.38,4.71,4.38,4.15,4.37,4.56,4.4,4.69
1,alfa-romeo giulia i,NaN,4.92,0.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,4.00,5.00,5.00,5.00,5.0,NaN
2,alfa-romeo junior suv-veloce-electric,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,alfa-romeo junior suv,NaN,4.07,NaN,3.00,5.00,5.00,4.00,5.00,2.00,3.00,5.00,5.00,5.00,4.00,1.00,5.0,5.00
4,alfa-romeo junior suv-electric,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [773]:
car_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3010 entries, 0 to 3009
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   model                      3010 non-null   object 
 1   new_price                  304 non-null    float64
 2   avg_rating                 2352 non-null   float64
 3   buy_again                  2195 non-null   float64
 4   overall_rating             2352 non-null   float64
 5   engine                     2352 non-null   float64
 6   gearbox                    2352 non-null   float64
 7   drivetrain                 2352 non-null   float64
 8   body                       2352 non-null   float64
 9   visibility                 2352 non-null   float64
 10  ergonomy                   2352 non-null   float64
 11  climate_control            2352 non-null   float64
 12  sound_insulation           2352 non-null   float64
 13  interior_space             2352 non-null   float

This one looks fine, just needs the "model" column split so it can create a primary key in the database.

In [793]:
name_lenghts = set()

for model in car_ratings['model']:
    name_lenghts.add(len(model.split(' ')))

print(name_lenghts)

{3}


Consistent naming, very nice. Previous function will work.

In [794]:
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

id_cols_ratings = ['brand', 'model', 'generation'] # These will become the primary key in the database
car_ratings[id_cols_ratings] = car_ratings['model'].apply(create_model_name_columns)

other_cols_ratings = [col for col in car_ratings.columns if col not in id_cols_ratings]
new_col_order_ratings = id_cols_ratings + other_cols_ratings
car_ratings = car_ratings[new_col_order_ratings]

car_ratings.head()

,brand,model,generation,new_price,avg_rating,buy_again,overall_rating,engine,gearbox,drivetrain,body,visibility,ergonomy,climate_control,sound_insulation,interior_space,maintenance_costs,quality_price_ratio,reliability_small_repairs,reliability_major_repairs
0,alfa-romeo,giulia,ii,232400.0,4.56,0.98,4.77,4.87,4.83,4.94,4.77,4.02,4.38,4.71,4.38,4.15,4.37,4.56,4.4,4.69
1,alfa-romeo,giulia,i,NaN,4.92,0.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,4.00,5.00,5.00,5.00,5.0,NaN
2,alfa-romeo,junior,suv-veloce-electric,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,alfa-romeo,junior,suv,NaN,4.07,NaN,3.00,5.00,5.00,4.00,5.00,2.00,3.00,5.00,5.00,5.00,4.00,1.00,5.0,5.00
4,alfa-romeo,junior,suv-electric,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
